1) Make a request from the Forecast.io API for where you were born (or lived, or want to visit!).

In [30]:
import requests

In [42]:
response = requests.get('https://api.forecast.io/forecast/2720ff846cb70aeec085c402d31c3ea2/40.7128,-74.0059')
data = response.json()

In [43]:
type(data)
data.keys()

dict_keys(['minutely', 'daily', 'timezone', 'latitude', 'currently', 'hourly', 'longitude', 'offset', 'flags'])

In [44]:
#Check the timezone
print(data['timezone'])
#Yay!
#I chose New York so I'll know when to bring my umbrella to class

America/New_York


2) What's the current wind speed? How much warmer does it feel than it actually is?

In [52]:
#Where is the wind speed? Maybe it's in 'currently' updates.
current = data['currently']
type(current)
current.keys()

dict_keys(['humidity', 'time', 'windBearing', 'windSpeed', 'temperature', 'precipType', 'precipIntensity', 'precipProbability', 'apparentTemperature', 'precipIntensityError', 'nearestStormDistance', 'dewPoint', 'cloudCover', 'icon', 'ozone', 'visibility', 'pressure', 'summary'])

In [66]:
#We need windspeed, temperature and apparent temperature
print("The current wind speed is", current['windSpeed'])

current_temp = current['temperature']
apparent_temp = current['apparentTemperature']
cold_temp = current_temp - apparent_temp
warm_temp = apparent_temp - current_temp

if current_temp - apparent_temp == 0:
    print("It feels as warm as it actually is.")
if current_temp - apparent_temp > 0:
    print("It currently feels", cold_temp, "degrees colder")
if current_temp - apparent_temp < 0:
    print("It currently feels", warm_temp, "degrees colder")

The current wind speed is 8.95
It feels as warm as it actually is.


3) The first daily forecast is the forecast for today. 
or the place you decided on up above, how much of the moon is currently visible?

#From the documentation: moonPhase (only defined on daily data points): A number representing the fractional part of the lunation number of the given day: a value of 0 corresponds to a new moon, 0.25 to a first quarter moon, 0.5 to a full moon, and 0.75 to a last quarter moon. (The ranges in between these represent waxing crescent, waxing gibbous, waning gibbous, and waning crescent moons, respectively.)

In [140]:
#We are looking for moonphase, which is only available in daily and represent weather that will
#occur over the entire day
daily = data['daily']
#print(daily)
daily.keys()
#print(daily['data'])
daily_info = daily['data']

#Looks like a list of dictionaries
#We have to look for the moonphase and the time

for info in daily_info:
    #print(info['moonPhase'], info['time'])
    if info['time'] == 1465358400:
        print(info['moonPhase'], "of the moon is currently visible")
    
#from datetime import datetime
#datetime.fromtimestamp(1465444800)

#okay I gotta find the human timings to see how much of the moon I can see now
#okay so these are daily timings, so I just need the first one

0.13 of the moon is currently visible


#Question: Why doesn't this work?
for info in daily_info:
    print(info['moonPhase'], info['time'])
    daily_time = info['time']
    from datetime import datetime
    print datetime.fromtimestamp('dailytime')

4) What's the difference between the high and low temperatures for today?

#Once again, according to the documentation: temperatureMin, temperatureMinTime, temperatureMax, and temperatureMaxTime (only defined on daily data points): numerical values representing the minimum and maximumum temperatures (and the UNIX times at which they occur) on the given day in degrees Fahrenheit.

In [151]:
for temp in daily_info:
    #print(temp['temperatureMin'], temp['temperatureMax'], temp['time'])
    temp_diff = temp['temperatureMax'] - temp['temperatureMin']
    if temp['time'] == 1465358400:
        print("The difference between the high and low temperatures for today is", temp_diff, "degrees")

The difference between the high and low temperatures for today is 10.740000000000002 degrees


5) Loop through the daily forecast, printing out the next week's worth of predictions. 
I'd like to know the high temperature for each day, and whether it's hot, warm, or cold, 
based on what temperatures you think are hot, warm or cold.

In [167]:
#Here's what we need: daily forecasts for the next week, 
#temperatureMax, my definition of hot and cold temperatures

for heat in daily_info:
    #print(heat['temperatureMax'], heat['time'])
    from datetime import datetime
    datetime = datetime.fromtimestamp(heat['time'])
    print("The high temperature for", datetime, "is", heat['temperatureMax'], "degrees")
    if heat['temperatureMax'] >= 80:
        print("Damn it's gonna be hot")
    if 70 <= heat['temperatureMax'] <= 79:
        print("I guess I'll wear a t-shirt because it's going to be warm")
    if heat['temperatureMax'] <= 69:
        print("Damn I hope they don't turn on the ac because it's going to be cold")

The high temperature for 2016-06-08 00:00:00 is 66.45 degrees
Damn I hope they don't turn on the ac because it's going to be cold
The high temperature for 2016-06-09 00:00:00 is 73.82 degrees
I guess I'll wear a t-shirt because it's going to be warm
The high temperature for 2016-06-10 00:00:00 is 76.5 degrees
I guess I'll wear a t-shirt because it's going to be warm
The high temperature for 2016-06-11 00:00:00 is 74.66 degrees
I guess I'll wear a t-shirt because it's going to be warm
The high temperature for 2016-06-12 00:00:00 is 77.77 degrees
I guess I'll wear a t-shirt because it's going to be warm
The high temperature for 2016-06-13 00:00:00 is 72.88 degrees
I guess I'll wear a t-shirt because it's going to be warm
The high temperature for 2016-06-14 00:00:00 is 75.21 degrees
I guess I'll wear a t-shirt because it's going to be warm
The high temperature for 2016-06-15 00:00:00 is 77.56 degrees
I guess I'll wear a t-shirt because it's going to be warm


6) What's the weather looking like for the rest of today in Miami, Florida? 
I'd like to know the temperature for every hour, and if it's going to have cloud cover of 
more than 0.5 say "{temperature} and cloudy" instead of just the temperature.

#Miami Coordinates: 25.7617° N, 80.1918° W
#From the documentation: cloudCover: A numerical value between 0 and 1 (inclusive) representing the percentage of sky occluded by clouds.

In [174]:
response = requests.get('https://api.forecast.io/forecast/2720ff846cb70aeec085c402d31c3ea2/25.7617,-80.1918')

In [172]:
miami = response.json()

In [208]:
#What I need: hourly, icon

miami.keys()
miami_hourly = miami['hourly']
miami_hourly.keys()
#print(miami_hourly['data'])
miami_hourly_data = miami_hourly['data']

for temp in miami_hourly_data:
    #print(temp['time'], temp['cloudCover'], temp['temperature'])
    from datetime import datetime
    todaytime = datetime.fromtimestamp(temp['time'])
    if temp['time'] < 1465444800 and temp['cloudCover'] <= 0.5:
        print("The temperature at", todaytime, "is", temp['temperature'], "degrees")
    if  temp['time'] < 1465444800 and temp['cloudCover'] > 0.5:
        print("The temperature at", todaytime, "is", temp['temperature'], "degrees and it is cloudy")

The temperature at 2016-06-08 17:00:00 is 81.19 degrees and it is cloudy
The temperature at 2016-06-08 18:00:00 is 80.76 degrees and it is cloudy
The temperature at 2016-06-08 19:00:00 is 80.59 degrees and it is cloudy
The temperature at 2016-06-08 20:00:00 is 79.96 degrees and it is cloudy
The temperature at 2016-06-08 21:00:00 is 78.8 degrees and it is cloudy
The temperature at 2016-06-08 22:00:00 is 78.02 degrees and it is cloudy
The temperature at 2016-06-08 23:00:00 is 78.14 degrees and it is cloudy


7) What was the temperature in Central Park on Christmas Day, 1980? How about 1990? 2000?

Tip: You'll need to use UNIX time, which is the number of seconds since January 1, 1970. Google can help you convert a normal date!
Tip: You'll want to use Forecast.io's "time machine" API at https://developer.forecast.io/docs/v2

In [214]:
response = requests.get('https://api.forecast.io/forecast/2720ff846cb70aeec085c402d31c3ea2/40.7829,-73.9654,346550400')
xmas_1980 = response.json()

In [216]:
response = requests.get('https://api.forecast.io/forecast/2720ff846cb70aeec085c402d31c3ea2/40.7829,-73.9654,662083200')
xmas_1990 = response.json()

In [217]:
response = requests.get('https://api.forecast.io/forecast/2720ff846cb70aeec085c402d31c3ea2/40.7829,-73.9654,662083200')
xmas_2000 = response.json()

In [240]:
#I need the temperature from current because it's not in daily
current_1980 = xmas_1980['currently']
#current_1980.keys()
current_1990 = xmas_1990['currently']
current_2000 = xmas_2000['currently']

print("The temperature on Christmas day at Central Park was", current_1980['temperature'], "degrees in 1980",
     current_1990['temperature'], "degrees in 1990 and", current_2000['temperature'], "degrees in 2000.")


The temperature on Christmas day at Central Park was 33.96 degrees in 1980 36.74 degrees in 1990 and 36.74 degrees in 2000.
